<a href="https://colab.research.google.com/github/xechehot/sirius-2021-warehouse/blob/main/Maksim/Correct_create_order.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json

In [ ]:
!git clone https://github.com/xechehot/sirius-2021-warehouse

fatal: destination path 'sirius-2021-warehouse' already exists and is not an empty directory.


In [ ]:
! cd sirius-2021-warehouse; git pull

Already up to date.


In [ ]:
from json import load as JLoad

with open('sirius-2021-warehouse/data_sample_output/output_64_1_Maksim.json', 'r') as f:
  data = JLoad(f) 

In [ ]:
with open('sirius-2021-warehouse/data_sample/sample_64_1.json', 'r') as f:
  sample_data = JLoad(f) 

In [ ]:
floors = sample_data['warehouse']['meta']['floors']
block_x = sample_data['warehouse']['meta']['block_x']
block_y = sample_data['warehouse']['meta']['block_y']
rows = sample_data['warehouse']['meta']['rows'] 
sections = sample_data['warehouse']['meta']['sections']

In [ ]:
def conv(place): 
    x = (sections + 2) * (place['block_x'] - 1)
    x += place['section']
    y = ((3 * rows) // 2 + 1) * (place['block_y'] - 1)
    y += (place['row'] // 2) * 3
    return [x, y]

In [ ]:
def conv_for_good(place):
    x = (sections + 2) * (place['block_x'] - 1)
    x += place['section']
    y = ((3 * rows) // 2 + 1) * (place['block_y'] - 1)
    y += (place['row'] // 2) * 3
    #print(y)
    if place['row'] % 2 == 0:
        y -= 1
    else: 
        y += 1
    return [x, y]

In [ ]:
def get_path_for_2_places(x1, y1, x2, y2):
    ans = []
    if y1 == y2:
        ans.append([x1, y1])
        ans.append([x2, y2])
        return ans
    ans.append([x1, y1])
    up_x = (x1 // map_x) * map_x
    ans.append([up_x, y1])
    ans.append([up_x, y2])
    ans.append([x2, y2])
    return ans


def normolize_path(path, start):
    path.append([0, path[-1][1]])
    path.append(start)
    res = []
    for i in range(1, len(path)):
        cur = get_path_for_2_places(path[i - 1][0], path[i - 1][1], path[i][0], path[i][1])
        for elem in cur:
            if len(res) == 0 or elem != res[-1]:
                res.append(elem)
    return res

In [ ]:
paths = []
map_y = ((3 * rows) // 2 + 1)
map_x = (sections + 2)
sx = block_x * map_x - 1
sy = block_y * map_y // 2
for batch in data['batches']:
    for bucket in batch['buckets']:
        #print(bucket)
        paths.append(bucket['items'])
ans = []
ans2 = []
for bucket in paths:
    #print(bucket)
    n_bucket = [[sx, sy]]
    items_bucket = []
    for place in bucket:
        n_bucket.append(conv(place))
    ans.append(n_bucket)
    ans2.append(bucket)
    #print(bucket)

output = {
    "agentsCount": 2,
    "orders": [

    ]
}

for i in range(len(ans)):
    bucket = ans[i]
    products = ans2[i]
    res = normolize_path(bucket, [sx, sy])
    bucket = {
        "items": [
                  
        ],
        "path": [
             
        ]
    }
    for place in products:
        x, y = conv_for_good(place)
        wtf = {
            "position": {
                "x": x,
                "y": y
            },
            "id": place['id'],
            "amount": place['cnt']
        }
        bucket['items'].append(wtf)
    for x, y in res:
        place = {
            "x": x,
            "y": y
        }
        bucket['path'].append(place)
    output['orders'].append(bucket)
#print(json.dumps(output, indent=2))
with open("visual_sample_order.json", "w") as f:
    json.dump(output, f)         